In [ ]:

# coding: utf-8

import pdfplumber
import pandas as pd
import os
import itertools #列表去除最外层 用

error = "" #错误的信息汇总

#读物文件家中的文件路径，返回列表
def file_name_paths(path=r'D:\UP'):
        path_collection=[]
        for dirpath,dirnames,filenames in os.walk(path):
                for file in filenames:
                        fullpath=os.path.join(dirpath,file)
                        path_collection.append(fullpath)
        return path_collection

#读取图纸中的表格，返回df格式
def srart(path):
    if '.pdf' in path:
        pdf = pdfplumber.open(path)
        n=0
        for i in pdf.pages:
            n = n+1
            if i.extract_text():
                if "设计站名"in i.extract_text():#表格中的“设计站名”关键字用来定位表格所在页
                    print('“',path,'” 在第',n,'页 ，读取成功  √')
                    table = i.extract_tables()
                    df=zhengli(table)
                    break
    
        if ('table' in locals()):
            return df
        else:
            print('“',path,'” 文件中无表格，读取失败  X')
            error_j='“'+path+'” 文件中无表格，读取失败  X\n'
            global error 
            error = error +error_j

#整理table中的数据，列表转换成df
def zhengli(table):
    if len(table)>1:
        df = pd.DataFrame(table[1:], columns=table[0])
    else:
        kk = list(itertools.chain(*table))
        df = pd.DataFrame(kk[1:], columns=kk[0])
    return df
#执行函数
def main(path=r'D:\up\tu',out_result='res.csv'):
    paths=file_name_paths(path=path)
    print('开始：一共',len(paths),'个文件,其中pdf文件',len([  x for x in paths if '.pdf'in x]),'个\n','_'*40)
    res = pd.DataFrame()
    for path_name in paths:
        res = res.append(srart(path_name))
    res['小区中文名']=res['小区中文名'].replace(r'\n','', regex=True)#过滤换行符号^^^^^^可根据需求修改
    if '.csv'in out_result:
        res.to_csv(out_result,encoding = 'gbk',index=False)
    else:
        res.to_excel(out_result,encoding = 'gbk',index=False)
    with open(path+r'\log.txt', "w", encoding='gbk') as f: #导出错误列表
        f.write(error)

if __name__ == '__main__':
    main(path=r'.',out_result=r'.\结果.xlsx') 
    print('结束')

